In [1]:
import requests
import fastf1

In [10]:
import datetime

In [2]:
fastf1.Cache.enable_cache("../tmp") 

In [60]:
df = fastf1.get_event_schedule(2022)[['RoundNumber', 'Country', 'EventDate', 'Location', 'EventFormat']]

In [61]:
df['has_happened'] = df['EventDate'] < datetime.datetime.now()
df = df.query('RoundNumber > 0')

In [62]:
df

,RoundNumber,Country,EventDate,Location,EventFormat,has_happened
2,1,Bahrain,2022-03-20 20:00:00,Sakhir,conventional,True
3,2,Saudi Arabia,2022-03-27 22:00:00,Jeddah,conventional,True
4,3,Australia,2022-04-10 17:00:00,Melbourne,conventional,True
5,4,Italy,2022-04-24 17:00:00,Imola,sprint,True
6,5,United States,2022-05-08 17:30:00,Miami,conventional,True
7,6,Spain,2022-05-22 17:00:00,Barcelona,conventional,True
8,7,Monaco,2022-05-29 17:00:00,Monaco,conventional,True
9,8,Azerbaijan,2022-06-12 17:00:00,Baku,conventional,True
10,9,Canada,2022-06-19 16:00:00,Montréal,conventional,True
11,10,Great Britain,2022-07-03 17:00:00,Silverstone,conventional,True


In [64]:
races = df.query('has_happened').Location.tolist()
formats = df.query('has_happened').EventFormat.tolist()

In [90]:
for n in range(len(races)-1, len(races)):

    temp = {}
    
    for sess in ['FP1', 'FP2', 'FP3']:

        if formats[n] == 'sprint' and sess == 'FP3':
            sess = 'Q'
        
        session = fastf1.get_session(2022, races[n], sess)
        session.load()
        
        if sess == 'Q':
            sess = 'FP3'

        drivers = session.results[['LastName', 'DriverNumber']]

        laptimes = session.laps.query('IsPersonalBest == True')[['DriverNumber', 'LapTime']]
        laptimes['rank'] = laptimes.LapTime.rank()

        temp[sess] = drivers.merge(laptimes, on='DriverNumber')[['LastName', 'rank']]
        temp[sess].columns = ['name', sess]
        
    out = temp['FP1'].merge(temp['FP2']).merge(temp['FP3'])
    out.columns = [x.lower() for x in out.columns]

    out.to_csv(f'fp/{n+1}.txt', sep=";", index=False)

core           INFO 	Loading data for Italian Grand Prix - Practice 1 [v2.2.7]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '55', '44', '34', '31', '24', '23', '22', '20', '18', '16', '14', '11', '10', '6', '4', '3', '77', '99']
core           INFO 	Loading data for Italian Grand Prix - Practice 2 [v2.2.7]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data fo

In [91]:
for n in range(len(races)-1, len(races)):

    session = fastf1.get_session(2022, races[n], 'R')
    session.load()
    
    drivers = session.results[['LastName', 'DriverNumber']]
    
    laptime = session.laps[['DriverNumber', 'LapNumber', 'Time']].query('LapNumber > 1')

    laptime['pos'] = laptime.groupby('LapNumber').Time.rank()
    
    temp = drivers.merge(laptime)[['LastName', 'LapNumber', 'pos']]
    temp.columns = ['last_name', 'Lap', 'pos']
    out = temp.pivot(index="lap", columns=['last_name'], values="pos").reset_index()
    
    out.to_csv(f'laps/{n+1}.txt', sep=",", index=False)

core           INFO 	Loading data for Italian Grand Prix - Race [v2.2.7]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '44', '11', '4', '10', '45', '24', '31', '47', '77', '22', '6', '20', '3', '18', '14', '5']


KeyError: 'lap'